![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook - Prompt Lab Notebook v1.1.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Prompt Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Defining parameters of the Model object
* Using the Model object to generate response using the defined model id, parameters and the prompt input

# Setup

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [ ]:
import os
from ibm_watsonx_ai import APIClient, Credentials
import getpass

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=getpass.getpass("Please enter your api key (hit enter): ")
)



# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [ ]:
model_id = "ibm/granite-3-8b-instruct"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [ ]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 2000,
    "min_new_tokens": 200,
    "stop_sequences": ["!","---"],
    "repetition_penalty": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [ ]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [ ]:
from ibm_watsonx_ai.foundation_models import ModelInference

model = ModelInference(
	model_id = model_id,
	params = parameters,
	credentials = credentials,
	project_id = project_id,
	space_id = space_id
	)


## Defining the inferencing input
Foundation model inferencing API accepts a natural language input that it will use
to provide the natural language response. The API is sensitive to formatting. Input
structure, presence of training steps (one-shot, two-shot learning etc.), as well
as phrasing all influence the final response and belongs to the emerging discipline of
Prompt Engineering.

Let us provide the input we got from the Prompt Lab:


In [ ]:
prompt_input = """You are a smart Travel Planner Agent.

Based on the following user input, generate a personalized 5-day itinerary with travel suggestions, accommodation, weather info (mock), and local attractions.

User input:
- Destination: {{destination}}
- Budget: {{budget}} INR
- Travel Month: {{month}}
- Number of People: {{people}}
- Interests: {{interests}}

Give output in this format:
- Destination Overview
- Day-wise Plan (Day 1 to Day 5)
- Estimated Budget Breakdown
- Mock Weather Summary
- Hotel & Transport Suggestions

always end Final notes with a "!"

Input: Destination: Jaipur  
Budget: 15000 INR  
Month: October  
People: 2  
Interests: culture, food, photography
Output: 🌆 Destination Overview:
Jaipur, the Pink City, is a historical hub known for its forts, palaces, and local Rajasthani cuisine.

📅 Day-wise Itinerary:
- Day 1: Arrival, check-in, visit Hawa Mahal and City Palace
- Day 2: Amber Fort & Elephant ride, evening local market
- Day 3: Food walk, visit Albert Hall Museum
- Day 4: Nahargarh Fort sunset, Chokhi Dhani dinner
- Day 5: Local shopping, depart

💸 Budget Breakdown:
- Travel: ₹3000
- Hotel (4 nights): ₹5000
- Food: ₹3000
- Entry fees & activities: ₹2000
- Buffer: ₹2000

🌦️ Weather:
Expected 25-30°C, mostly clear skies

🏨 Suggested Hotel:
Hotel Pearl Palace (3-star, ₹1200/night)

🚌 Transport:
Local cab + auto rickshaw combo

📝 Final Notes:
Best to travel with light clothes, camera ready. Book tickets for Amber Fort early
---

Input: "I’m planning a 5-day trip to Jaipur this October with one other person. Our total budget is ₹15,000. We’re really interested in exploring culture, trying local food, and taking lots of great photos. Can you suggest a personalized travel itinerary that fits our interests and budget?"
Output: 🌆 Destination Overview:
Jaipur, the Pink City, is a historical hub known for its forts, palaces, and local Rajasthani cuisine.

📅 Day-wise Itinerary:
- Day 1: Arrival, check-in, visit Hawa Mahal and City Palace
- Day 2: Amber Fort & Elephant ride, evening local market
- Day 3: Food walk, visit Albert Hall Museum
- Day 4: Nahargarh Fort sunset, Chokhi Dhani dinner
- Day 5: Local shopping, depart

💸 Budget Breakdown:
- Travel: ₹3000
- Hotel (4 nights): ₹5000
- Food: ₹3000
- Entry fees & activities: ₹2000
- Buffer: ₹2000

🌦️ Weather:
Expected 25-30°C, mostly clear skies

🏨 Suggested Hotel:
Hotel Pearl Palace (3-star, ₹1200/night)

🚌 Transport:
Local cab + auto rickshaw combo

📝 Final Notes:
Best to travel with light clothes, camera ready. Book tickets for Amber Fort early.
---

Input: "Hey! I’m planning a solo getaway to Goa this December. I’ve got a budget of around ₹25,000 and I’m looking for a fun mix of beaches, nightlife, and maybe some nature spots too. Can you help me come up with a 5-day travel plan that balances relaxation and adventure?"
Output: 🏖️ Travel Itinerary: Solo Trip to Goa (December, ₹25,000 Budget)
🌍 Destination Overview
Goa is India’s ultimate coastal paradise—famous for its golden beaches, vibrant nightlife, and lush nature escapes. December is the peak season, offering perfect weather and a festive atmosphere.

📅 5-Day Itinerary
🗓️ Day 1 – Arrival & Chill

Check-in at a budget-friendly hostel in North Goa (Zostel or The Hosteller)

Relax at Anjuna Beach

Sunset at Chapora Fort

Dinner at Curlies Beach Shack
💰 Approx. ₹3,500

🗓️ Day 2 – Beach Hop & Party Vibes

Morning: Baga Beach & water sports (banana ride, jet ski)

Lunch at Britto’s

Evening: Club hopping at Titos Lane (Cafe Mambo, LPK)
💰 Approx. ₹4,500

🗓️ Day 3 – Culture + Nature

Visit Old Goa Churches (Basilica of Bom Jesus, Sé Cathedral)

Explore the Latin Quarter in Panjim

Mandovi River cruise in the evening
💰 Approx. ₹3,000

🗓️ Day 4 – South Goa Serenity

Day trip to Palolem & Agonda Beaches

Dolphin spotting boat ride

Dinner at Art Resort Cafe
💰 Approx. ₹4,000

🗓️ Day 5 – Shopping & Departure

Morning flea market shopping at Arpora

Brunch and chill at a beach cafe

Return flight/train
💰 Approx. ₹3,000

💸 Budget Breakdown (₹25,000)
Category	Approx. Cost
Stay (4 nights)	₹5,000
Food & Drinks	₹4,500
Activities & Entry Fees	₹5,500
Local Transport (Scooter + Taxi)	₹3,000
Shopping + Misc	₹2,000
Travel (Flights/Train)	₹5,000

🌤️ Weather in December
Avg Temp: 28–30°C

Pleasant & dry, ideal for beach activities

🏨 Stay Suggestion
Zostel Anjuna or The Hosteller (₹1,000–1,200/night)

🚗 Transport
Rent a scooter (₹350/day) or hire local taxis as needed

📝 Final Notes
Book clubs and water activities early (they sell out fast in December)

Keep cash for flea markets

Carry sunscreen and light cotton clothing

Be ready to make new friends — Goa is great for solo travelers!
---


Input: I'm planning a solo 5-day trip to Budapest, Hungary next month. My budget is ₹60,000 (around €650). I enjoy historical sites, street food, scenic walks, and photography. Can you create a detailed 5-day itinerary that includes:

- Recommended places to visit
- A simple daily schedule
- Accommodation and transport suggestions
- Approximate daily budget breakdown

Output:"""


## Execution
Let us now use the defined Model object and pair it with input and
generate the response:


In [ ]:
print("Submitting generation request...")
generated_response = model.generate_text(prompt=prompt_input, guardrails=True)
print(generated_response)


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  